In [4]:
pip install trtokenizer

In [3]:
pip install TurkishStemmer

In [15]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from snowballstemmer import stemmer
from gensim.models import FastText
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from TurkishStemmer import TurkishStemmer



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_excel('tweetData.xlsx')

In [7]:
df.shape

(15604, 2)

In [8]:
df = df.dropna()

In [9]:
df.shape

(1976, 2)

In [10]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [11]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [13]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [17]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []

# Stop kelimeleri temizleme işlevi
def remove_stopwords_nltk(words):
    stop_words = set(stopwords.words('turkish'))
    return [word for word in words if word.lower() not in stop_words]

def turkishstemmer(text):
  stemmer = TurkishStemmer()
  # Stem işlemi
  stemmed_words = [stemmer.stem(word) for word in text]
  return ' '.join(stemmed_words)



# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_nltk)
df['lemmatized_words'] = df['cleaned_words'].apply(turkishstemmer)

# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [18]:

# Kelimeleri tokenize et
tokenize_metni = [metin.lower().split() for metin in df['lemmatized_words']]

# FastText modelini eğit
fasttext_model = FastText(vector_size=100, window=3, min_count=1, epochs=10)
fasttext_model.build_vocab(corpus_iterable=tokenize_metni)
fasttext_model.train(corpus_iterable=tokenize_metni, total_examples=len(tokenize_metni), epochs=10)

# Her metin için ortalama kelime vektörlerini hesapla
def ortalama_vektor(metin):
    kelimeler = metin.lower().split()
    vektörler = [fasttext_model.wv[kelime] for kelime in kelimeler if kelime in fasttext_model.wv]
    if vektörler:
        return np.mean(vektörler, axis=0)
    else:
        return np.zeros(100)

df['fasttext_vektor'] = df['lemmatized_words'].apply(ortalama_vektor)
print(df['fasttext_vektor'] )



0        [-0.5206835, -0.35515457, -0.09953377, 0.67032...
1        [-0.597314, -0.40558323, -0.11379113, 0.765821...
2        [-0.39746663, -0.2703361, -0.07479606, 0.51036...
3        [-0.63241875, -0.43057862, -0.12080735, 0.8130...
4        [-0.43910012, -0.29919687, -0.08339468, 0.5646...
5        [-0.5392118, -0.36677554, -0.102241576, 0.6909...
6        [-0.5522052, -0.37579054, -0.104817085, 0.7098...
7        [-0.66378284, -0.45154083, -0.12587737, 0.8519...
8        [-0.63220257, -0.43115208, -0.12185262, 0.8117...
9        [-0.64315975, -0.43777516, -0.12251762, 0.8264...
10       [-0.45665693, -0.3107901, -0.086614825, 0.5863...
11       [-0.4957377, -0.3383669, -0.09482337, 0.637585...
12       [-0.56594324, -0.385003, -0.10836413, 0.728281...
13       [-0.5298628, -0.35920617, -0.10075427, 0.68194...
14       [-0.5127635, -0.34827292, -0.09814343, 0.65730...
15       [-0.47186705, -0.32094738, -0.08942477, 0.6056...
16       [-0.5378705, -0.366152, -0.103374444, 0.691937.